In [1]:
import pandas as pd
import pickle 
import xml.etree.cElementTree as ET
from bs4 import BeautifulSoup
import requests
import urllib
import os
import zipfile
import lxml

%load_ext autoreload
%autoreload 2
#import patent_utils as pu

In [4]:
%ls

APPLICATION_XML_FILE_INSPECTION.ipynb  data/
COMBINING_ALL_DATA.ipynb               data-checking.ipynb
GRANTS_XML_PREPARATION.ipynb           data_collection.ipynb
__pycache__/                           patent_utils.py
check-citations-data.ipynb


# 1. Load Office Actions/Application/Grants


In [5]:
%%time
grants = pd.read_csv('/work/data/grants/2012_all_grants.csv').iloc[:, 1:]
apps = pd.read_csv('/work/data/apps/2017_all_apps.csv').iloc[:, 1:]
oas = pd.read_csv('/work/data/office_actions.csv')
cits = pd.read_csv('/work/data/citations.csv')
rej = pd.read_csv('/work/data/rejections.csv')

grants['pub_doc-number'].loc[grants['pub_doc-number'].str.startswith('0')] = \
grants['pub_doc-number'].loc[grants['pub_doc-number'].str.startswith('0')].\
    astype(int).astype(str)
grants = grants.rename(columns = {'pub_doc-number': 'parsed'})
apps = apps.rename(columns = {'app_doc-number': 'app_id'})


<string>:2: DtypeWarning: Columns (1,4,5) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 1min 40s, sys: 16.1 s, total: 1min 56s
Wall time: 2min 18s


/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
xa = apps.loc[:, ['app_id', 'fnm']].rename(columns = {'fnm':'app_fnm'}).merge(
    cits, on='app_id', how='inner').merge(grants, on='parsed', how='inner')
xa.loc[~xa.ifw_number.isnull()].merge(oas, on=['app_id', 'ifw_number'], 
                                      how='inner').app_id.unique().shape

(471,)

# 2. Get a sense about data coverage/size
- 2 Mn unique app_ids with citations
- 5 Mn citations in total
- 1.9 Mn citations with office actions in total

In [ ]:
oas.merge(cits, on=['app_id', 'ifw_number'], how='inner').app_id.unique().shape

In [ ]:
%ls data/apps

In [8]:
apps = [pd.read_csv('/work/data/apps/'+x).iloc[:, 1:] for x in 
            os.listdir('/work/data/apps') if '.csv' in x]
apps = pd.concat(apps)

grants = [pd.read_csv('/work/data/grants/'+x).iloc[:, 1:] for x in 
             os.listdir('/work/data/grants') if '.csv' in x]
grants = pd.concat(grants)

grants['pub_doc-number'].loc[grants['pub_doc-number'].str.startswith('0')] = \
grants['pub_doc-number'].loc[grants['pub_doc-number'].str.startswith('0')].\
    astype(int).astype(str)
grants = grants.rename(columns = {'pub_doc-number': 'parsed'})
apps = apps.rename(columns = {'app_doc-number': 'app_id'})


/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
xa = apps.loc[:, ['app_id', 'fnm']].rename(columns = {'fnm':'app_fnm'}).merge(
    cits, on='app_id', how='inner').merge(grants, on='parsed', how='inner')
xa.loc[~xa.ifw_number.isnull()].merge(oas, on=['app_id', 'ifw_number'], 
                                      how='inner').app_id.unique().shape

(494374,)

In [ ]:
xa.shape

In [ ]:
xa.loc[~xa.ifw_number.isnull()].merge(oas, on=['app_id', 'ifw_number'], 
                                      how='inner').app_id.unique().shape

In [ ]:
grants.head()

In [ ]:
apps_df.head()

In [ ]:
apps_df.head()

In [ ]:
pd.read_csv('data/apps/2012_all_apps.csv').iloc[:, 1:]

In [ ]:
print(apps.app_id.shape[0] - apps.app_id.unique().shape[0])
apps = apps.loc[apps.reset_index().groupby('app_id')['app_date'].idxmax()]
print(apps.app_id.shape[0] - apps.app_id.unique().shape[0])
apps = pd.concat([apps.set_index('app_id')], axis=1, keys=['APPLICATION'])

In [ ]:
cits = pd.concat([cits.set_index(['app_id', 'ifw_number', 'parsed'])],
                  axis=1, keys=['CITATIONS'])

In [ ]:
cits_17 = cits.join(apps, how='inner')
cits_17.shape

In [ ]:
oas_17 = oas.set_index(['app_id', 'ifw_number']).join(apps, how='inner')

In [ ]:
oas = pd.concat([oas.set_index(['app_id', 'ifw_number'])],
                  axis=1, keys=['OFFICE ACTIONS'])
oas_17 = oas.join(apps, how='inner')
oas_17.shape

# Brief summary
- 2017 Applications have 300K citations
- 2017 Applications have around 30K of office actions

In [ ]:
print(grants.parsed.shape[0] - grants.parsed.unique().shape[0])

In [ ]:
grants = pd.concat([grants.set_index('parsed')], axis=1, keys=['GRANTS'])
cits_12_17 = cits_17.join(grants, how='inner')
cits_12_17 = cits_12_17.reset_index('ifw_number').drop('ifw_number', axis=1).\
           drop_duplicates()
cits_12_17.shape

In [ ]:
cits_12_17.drop_duplicates().shape

In [ ]:
cits.index.get_level_values(1).isnull().sum()/cits.index.get_level_values(1).isnull().sum()

In [ ]:
oasifw = oas.merge(cits, on=['app_id', 'ifw_number'], how='left')
oasifw = oasifw.dropna(subset=['parsed'])

In [ ]:
cits_2017 = cits.merge(apps, on='app_id')
cits_2017.shape

In [ ]:
oas_2017 = oas.merge(apps, on='app_id')
oas_2017.shape

In [ ]:
cits_12_17 = cits_2017.merge(grants.rename(columns = {'pub_doc-number': 'parsed'}).\
                astype('str'), on='parsed')

In [ ]:
cits_12_17.parsed.unique().shape
cits_12_17.head()

In [ ]:
cits_12_17.merge(oas_2017, on=['app_id', 'ifw_number']).shape

In [ ]:
oapps = apps.rename(columns = {'app_doc-number':'app_id'}).\
             merge(oasifw, on='app_id')

In [ ]:
oappsgr = oapps.merge(grants.rename(columns = {'pub_doc-number':'parsed'}),
                      on='parsed')

In [ ]:
pgga = pd.read_pickle('/work/data/patent_grants_dic_grants12_app17.dat')b


In [ ]:
oappsgr.loc[:, ['app_id', 'parsed']].drop_duplicates().dropna().shape

In [ ]:
lpgga = list(pgga.keys())

In [ ]:
pgga[lpgga[0]]

In [ ]:
oappsgr.drop_duplicates().shape

In [ ]:
len(pgga.keys())

In [ ]:
oappsgr.info()

In [ ]:
oapps.parsed.isin([])

In [ ]:
grants.loc[grants['pub_doc-number'].isin(['5425307'])]

In [ ]:
grants['pub_doc-number'].loc[grants['pub_doc-number'].str.startswith('0')] = \
grants['pub_doc-number'].loc[grants['pub_doc-number'].str.startswith('0')].\
    astype(int).astype(str)

In [ ]:
grants.pub_doc-number.

In [ ]:
cits_12

In [ ]:
oapps.info()

In [ ]:
oasifw.shape

In [ ]:
oasifw.dropna(subset=['parsed']).shape

In [ ]:
cits.loc[cits.ifw_number == '100867762']

In [ ]:
oasifw.head()

In [ ]:
cits.ifw_number.loc[~cits.ifw_number.isnull()]

In [ ]:
%ls /work/data/

# Load grant and application data from 2005
